## Abrir archivo y estudiar la informacion general

In [ ]:
# Cargamos la librerias para dataframe

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Lectura del archivo

df = pd.read_csv('/datasets/games.csv')


In [ ]:
# Tamaño deñ dataset

print("Shape del DataFrame:", df.shape)


In [ ]:
# Revision de los nombres de las columnas

print('\nColumnas del DataFrame:')
print(df.columns)

In [ ]:
# Revision de primeras filas del DataFrame

print('\nPrimeras 5 filas del dataframe:')
print(df.head(5))


In [ ]:
# Informacion general de columnas y tipos de datos

print('\nInformacion del DataFrame:')
df.info()


In [ ]:
# Conteo de valores ausentes por columnas

print("\nValores ausentes por columna:")
print(df.isna().sum())


## Conclusion paso 1

En este primer paso cargamos el dataset 'games.csv' con información de **16,715 filas y 11 columnas** sobre videojuegos: nombre, plataforma, año de lanzamiento, género, ventas por región, puntuaciones de usuarios y críticos, y clasificación ESRB.  

Identificamos que varias columnas presentan valores ausentes (`Year_of_Release`, `Critic_Score`, `User_Score`, `Rating`) y que algunos tipos de datos no son los más adecuados (por ejemplo, `Year_of_Release` como *float*, `User_Score` como *object*).  

Con esta exploración inicial, ya estamos listos para el **Paso 2: Preparar los datos**, donde vamos a:  
- Estandarizar nombres de columnas en minúsculas.  
- Corregir los tipos de datos según corresponda.  
- Manejar valores ausentes y casos con `'tbd'`.  
- Crear una columna nueva con las **ventas totales** por juego.  


## Preparacion de los datos para analisis posterior

In [ ]:
# Convertir los nombres de las columnas a minusculas

df.columns = df.columns.str.lower()

# Verificamos que el cambio se haya hecho

print(df.columns)


In [ ]:
# Revisamos los tipos de datos actuales despues del cambio de nombres

print(df.dtypes)


In [ ]:

# Serie de trabajo

s = df['user_score']

# Conteos básicos

nan_count  = s.isna().sum()
tbd_count  = (s == 'tbd').sum()

# Conteo de numéricos (convirtiendo 'tbd' a NaN y forzando errores a NaN)

s_num = pd.to_numeric(s.replace('tbd', np.nan), errors='coerce')
numeric_count = s_num.notna().sum()

# Totales y verificación de informacion

total = s.shape[0]
other_non_numeric = total - (nan_count + tbd_count + numeric_count)

print("Total filas en user_score:", total)
print("Total filas NaN:", nan_count)
print("Total filas 'tbd':", tbd_count)
print("Total filas Numéricos:", numeric_count)
print("Otros valores NO numéricos distintos de NaN/'tbd':", other_non_numeric)


## La suma de estos valores coincide con el total de filas de la columna (Tot = NaN + 'tbd' + Numericos), lo que confirma que no existen otros valores no numéricos distintos de `'tbd'` y `NaN`. Por lo tanto, el procedimiento adecuado es reemplazar `'tbd'` por `NaN` y luego convertir toda la columna a tipo `float64`.

In [ ]:
# Reemplazar 'tbd' con NaN

df['user_score'] = df['user_score'].replace('tbd', np.nan)

# convertir a numerico (float64), ignorando errores

df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

print(df['user_score'].dtypes)




In [ ]:
# Convertir a int64 (para permitir NaN como int)

df['year_of_release'] = df['year_of_release'].astype('Int64')

print(df['year_of_release'].dtypes)

In [ ]:
# Confirmamos tipo de datos en critic_score

print(df['critic_score'].dtypes)

# Conversion de tipos

'User Score': se convirtio a float64 tras reemplazar 'tbd' por NaN.
'Year_of_release': convertido a Int64 para manejar enteros y NaN.
'Critic_score': ya estaba en float64, no requirio cambios.


In [ ]:
# Revision rapida de ausentes en columnas clave

cols_check = ['year_of_release', 'critic_score', 'user_score', 'rating']

print(df[cols_check].isna().sum())

In [ ]:
# EN algunos datasets 'rating' puede traer 'tbd'; lo convertimos a NaN por consistencia

if df['rating'].dtype == 'object':
    df['rating'] = df['rating'].replace('tbd', np.nan)

## Desicion de proyecto:

Para las columnas de year_of_release (Int64), critic_score (float), user_score (float), 
rating (object) dejamos NaN para evitar inventar datos y porque pandas ignora los NaN de 
forma segura en las estadisticas, con esto creemos que los datos seran mas exactos en los analisis posteriores, tambien filtraremos por año solo cuando el analisis lo requiera.  

In [ ]:
# Crear columna de ventas totales por juego (suma de regiones)

sales_cols = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
df['total_sales'] = df[sales_cols].sum(axis=1)

# Vista rapida para validar

print(df[['name', 'platform', 'year_of_release', 'total_sales']].head(5))

# Comprobacion

print("\nVerificar NaN en ventas:")
print(df[sales_cols + ['total_sales']].isna().sum())

## Coclusion Valores Ausentes y totales

Mantenemos NaN en year_of_release, critic_score, user_score, rating
Normalizamos 'tbd' como NaN (Tambien en rating si aparecia)
Creamos total_sales como suma de ventas por region


## Analisis de datos

In [ ]:
# Conteo de juegos lanzados por año 

releases_per_year = (df.loc[df['year_of_release'].notna(), 'year_of_release'].value_counts().sort_index())

print(releases_per_year)

In [ ]:
# Grafica de lanzamientos por año

releases_per_year.plot(kind='line', figsize=(10,4))
plt.title('Juegos lanzados por año')
plt.xlabel('Año')
plt.ylabel('Cantidad de juegos')
plt.show()

In [ ]:
# Sumamos ventas totales por plataforma (toda la serie histórica)

platform_totals = (
    df.groupby('platform', dropna=False)['total_sales']
      .sum()
      .sort_values(ascending=False)
)

print(platform_totals.head(10))




In [ ]:
# Elegimos los 5 plataformas con mas venta acumulada

top_platforms = platform_totals.head(5).index.tolist()

print('Top plataformas por ventas acumuladas:', top_platforms)

In [ ]:
# Filtrado por año

df_year = df[df['year_of_release'].notna()].copy()

# Suma de ventas por plataforma y año

sales_year_platform = (
    df_year.groupby(['platform', 'year_of_release'])['total_sales']
    .sum().reset_index().sort_values(['platform', 'year_of_release'])
)

print(sales_year_platform.head(10))

In [ ]:
# Graficamos informacion para mejor visualizacion

plt.figure(figsize=(10,5))

for plat in top_platforms:
    sub = sales_year_platform[sales_year_platform['platform'] == plat]
    plt.plot(sub['year_of_release'], sub['total_sales'], label=plat)

    plt.title('Ventas por año - plataformas TOP')
    plt.xlabel('Año')
    plt.ylabel('Ventas totales (millones USD)')
    plt.legend()
    plt.show()
    

In [ ]:
# Tomamos años de primera y ultima venta > 0

life = (df_year[df_year['total_sales'] > 0]
       .groupby('platform')['year_of_release']
       .agg(first_year='min', last_year='max', count='count')
       .reset_index())

life['active_2016'] = (life['last_year'] == 2016)
retiradas = life[~life['active_2016']].sort_values('last_year')

print('Plataformas retiradas en 2016:')
print(retiradas.head(25))

print('\nResumen vida por plataforma:')
print(life.sort_values('first_year').head(31))


In [ ]:
# Vida util aproximada

life['lifespan_years'] = life['last_year'] - life['first_year']

print("Vida útil (años) - estadísticos simples")
print(life['lifespan_years'].describe()[['count', 'min', '25%', '50%', 'mean', '75%', 'max']])


In [ ]:
# Conteo oir año

print('Lanzamientos por año (mas recientes)')
print(releases_per_year.tail(10))

In [ ]:
# Confirmar tipos y conteos por año

print("dtype year_of_release:", df['year_of_release'].dtype)

counts = (
    df.loc[df['year_of_release'].notna(), 'year_of_release']
      .value_counts()
      .sort_index()
)
print("\nConteo 2010–2016 en df base:")
for y in range(2010, 2017):
    print(y, counts.get(y, 0))

# df_actual desde df base 2013-2016

min_year = 2013  
mask = df['year_of_release'].notna() & (df['year_of_release'] >= min_year)
df_actual = df.loc[mask].copy()

print("\nRango en df_actual:",
      int(df_actual['year_of_release'].min()),
      "-",
      int(df_actual['year_of_release'].max()))
print("Tamaño:", df_actual.shape)

print("\nConteos en df_actual por año:")
print(df_actual['year_of_release'].value_counts().sort_index())


In [ ]:
# Top plataformas del periodo elegido

platform_totals_actual = (df_actual.groupby('platform')['total_sales'].sum().sort_values(ascending=False))

print('Top 10 plataformas (2013-2016):')
print(platform_totals_actual.head(10))


In [ ]:
# Guardado de listas de plataformas top

top_platforms_actual =platform_totals_actual.head(5).index.tolist()

print('Plataformas Top recientes:', top_platforms_actual)

In [ ]:
# Ventas por año y plataforma para las Top en df_actual

sales_year_top = (
    df_actual[df_actual['platform'].isin(top_platforms_actual)]
    .groupby(['platform', 'year_of_release'])['total_sales']
    .sum().reset_index().sort_values(['platform', 'year_of_release'])
)

print(sales_year_top.head(20))

In [ ]:
# grafica de tendencias por año

plt.figure(figsize=(10,5))
for plat in top_platforms_actual:
    sub = sales_year_top[sales_year_top['platform'] == plat]
    plt.plot(sub['year_of_release'], sub['total_sales'], label=plat)
plt.title('Ventas por año — plataformas TOP (ventana reciente)')
plt.xlabel('Año')
plt.ylabel('Ventas totales (millones USD)')
plt.legend()
plt.show()

In [ ]:
# Pivote para comparar 2016 vs 2015 

pivot = sales_year_top.pivot(index='platform', columns='year_of_release', values='total_sales').fillna(0)
for y in [2015, 2016]:
    if y not in pivot.columns:
        pivot[y] = 0
pivot['delta_2016_2015'] = pivot[2016] - pivot[2015]

print("Crecen más:")
print(pivot[['delta_2016_2015']].sort_values('delta_2016_2015', ascending=False).head(10))
print("\nCaen más:")
print(pivot[['delta_2016_2015']].sort_values('delta_2016_2015', ascending=True).head(10))


In [ ]:
# Boxplot de ventas globales por plataforma

subset = df_actual[df_actual['platform'].isin(top_platforms_actual)]
data = [subset[subset['platform'] == p]['total_sales'] for p in top_platforms_actual]

plt.figure(figsize=(10,5))
plt.boxplot(data, labels=top_platforms_actual, showfliers=False)
plt.title('Distribución de ventas globales por plataforma (ventana reciente)')
plt.ylabel('Ventas (millones USD)')
plt.show()


In [ ]:
# Ventas promedio por plataforma

print(subset.groupby('platform')['total_sales'].mean().sort_values(ascending=False))

In [ ]:
# Seleccion de la plataforma para el analisis de reseñas

print("Plataformas TOP recientes:", top_platforms_actual)
plat_sel = top_platforms_actual[0]   
print("Plataforma seleccionada:", plat_sel)


In [ ]:
# Eleccion de TOP con mas reseñas

plat_sel = (
    df_actual[df_actual['platform'].isin(top_platforms_actual)]
      .groupby('platform')['name']
      .count()
      .sort_values(ascending=False)
      .index[0]
)
print("Plataforma seleccionada por cantidad de juegos:", plat_sel)

In [ ]:
# Elegimos la plataforma 

print("Plataformas TOP recientes:", top_platforms_actual)
plat_sel = top_platforms_actual[0]
print("Plataforma seleccionada:", plat_sel)

# Subconjunto con columnas necesarias

cols_needed = ['name', 'platform', 'total_sales', 'user_score', 'critic_score']
sub = df_actual[df_actual['platform'] == plat_sel][cols_needed].copy()

# Quitamos filas sin ventas o sin reseñas

sub = sub.dropna(subset=['total_sales', 'user_score', 'critic_score'])

print("Filas para correlación en", plat_sel, ":", sub.shape[0])
print(sub.head(5))


In [ ]:
# Grafico de dispersion user_score vs total_sales

plt.figure(figsize=(6,4))
plt.scatter(sub['user_score'], sub['total_sales'], alpha=0.6)
plt.title(f'User score vs ventas - {plat_sel}')
plt.xlabel('User score (0-10)')
plt.ylabel('Ventas totales (millones USD)')
plt.show()

In [ ]:
# Grafico de dispersion critic_score vs total_sales

plt.figure(figsize=(6,4))
plt.scatter(sub['critic_score'], sub['total_sales'], alpha=0.6)
plt.title(f'Critic score vs ventas — {plat_sel}')
plt.xlabel('Critic score (0–100)')
plt.ylabel('Ventas totales (millones USD)')
plt.show()


In [ ]:
# Correlaciones (pearson) con ventas

corr_user = sub[['user_score', 'total_sales']].corr().loc['user_score', 'total_sales']
corr_crit = sub[['critic_score', 'total_sales']].corr().loc['critic_score','total_sales']

print(f"Correlación user_score–ventas ({plat_sel}): {corr_user:.3f}")
print(f"Correlación critic_score–ventas ({plat_sel}): {corr_crit:.3f}")


In [ ]:
# Comparar los mismos juegos en otras plataformas

name_sel = sub['name'].unique()

same_games = (
    df_actual[df_actual['name'].isin(name_sel)]
    .groupby(['name', 'platform'])['total_sales']
    .sum()
    .reset_index()
    .sort_values(['name', 'total_sales'], ascending=[True, False])
)

print(same_games.head(15))

In [ ]:
# Casos en donde otra plataforma supera a la seleccionada

ranked = same_games.assign(
    rank = same_games.groupby('name')['total_sales'].rank(method='first', ascending=False)
)

otros_ganan = ranked[(ranked['rank'] == 1) & (ranked['platform'] != plat_sel)]

print("Juegos donde otra plataforma supera a", plat_sel, "(muestra):")
print(otros_ganan.head(10))


## Conclusiones 

Periodo analizado: 2013–2016 (volumen estable; 2016 cae un poco).

Top plataformas por ventas (2013–2016): PS4 (314.14), PS3 (181.43), XOne (159.32), 3DS (143.25), X360 (136.80).  
Tendencia 2016 vs 2015: todas caen (PS4 −49.65, XOne −33.99, etc.).

Distribución de ventas por juego: media > mediana en todas > pocos “hits” tiran del promedio.

Reseñas vs ventas (PS4): 
user_score: sin relación (−0.034).  
critic_score: relación positiva moderada (+0.406).

Mismo juego en otras plataformas:~16.1% vende más fuera de PS4 > la plataforma importa por título.

Recomendación 2017: 
Priorizar PS4/XOne por escala reciente (solo es conveniente revisar la caida del 2016).  
Enfocar marketing en títulos con buenas críticas(señal de ventas).  
Evaluar multi-plataforma según cada juego.

# Perfil por region 

In [ ]:
# Top 5 plataformas por region (NA)

region_df = df_actual.copy()
region = 'na_sales'

na_plat = (
    region_df.groupby('platform')[region]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .reset_index()
)

total_na = region_df[region].sum()
na_plat['share_pct'] = (na_plat[region] / total_na) * 100

print('Top 5 plataformas -. NA')
print(na_plat)


In [ ]:
# Top 5 generos por region (NA)


# ESRB vs ventas (NA) + gráfica de barras
region_df = df_actual.copy()
region = 'na_sales'

na_rating = (
    region_df.dropna(subset=['rating'])
             .groupby('rating')[region]
             .sum()
             .sort_values(ascending=False)
)

# Tabla
print("Ventas por rating ESRB — NA")
print(na_rating.reset_index())


In [ ]:
# Top 5 plataformas por region (EU)

region = 'eu_sales'

eu_plat = (
    region_df.groupby('platform')[region]
             .sum()
             .sort_values(ascending=False)
             .head(5)
             .reset_index()
)
total_eu = region_df[region].sum()
eu_plat['share_pct'] = (eu_plat[region] / total_eu) * 100

print("Top 5 plataformas — EU")
print(eu_plat)


In [ ]:
# Top 5 generos por region (EU)

eu_genre = (
    region_df.groupby('genre')[region]
             .sum()
             .sort_values(ascending=False)
             .head(5)
             .reset_index()
)
eu_genre['share_pct'] = (eu_genre[region] / total_eu) * 100

print("Top 5 géneros — EU")
print(eu_genre)


In [ ]:
# ESBR Vs ventas (EU)

eu_rating = (
    region_df.dropna(subset=['rating'])
             .groupby('rating')[region]
             .sum()
             .sort_values(ascending=False)
             .reset_index()
)
eu_rating['share_pct'] = (eu_rating[region] / total_eu) * 100

print("Ventas por rating ESRB — EU")
print(eu_rating.head(10))


In [ ]:
# Top 5 Plataformas por region (JP)

region = 'jp_sales'

jp_plat = (
    region_df.groupby('platform')[region]
             .sum()
             .sort_values(ascending=False)
             .head(5)
             .reset_index()
)
total_jp = region_df[region].sum()
jp_plat['share_pct'] = (jp_plat[region] / total_jp) * 100

print("Top 5 plataformas — JP")
print(jp_plat)


In [ ]:
# Top 5 generos por region (JP)

jp_genre = (
    region_df.groupby('genre')[region]
             .sum()
             .sort_values(ascending=False)
             .head(5)
             .reset_index()
)
jp_genre['share_pct'] = (jp_genre[region] / total_jp) * 100

print("Top 5 géneros — JP")
print(jp_genre)


In [ ]:
# ESBR Vs Ventas (JP)

jp_rating = (
    region_df.dropna(subset=['rating'])
             .groupby('rating')[region]
             .sum()
             .sort_values(ascending=False)
             .reset_index()
)
jp_rating['share_pct'] = (jp_rating[region] / total_jp) * 100

print("Ventas por rating ESRB — JP")
print(jp_rating.head(10))


In [ ]:
# NA top 5 generos 

region_df = df_actual.copy()
region = 'na_sales'

na_genre = (
    region_df.groupby('genre')[region]
             .sum()
             .sort_values(ascending=False)
             .head(5)
             .reset_index()
)
total_na = region_df[region].sum()
na_genre['share_pct'] = (na_genre[region] / total_na) * 100
print("Top 5 géneros — NA")
print(na_genre)


In [ ]:
# NA - esbr con % y barra

na_rating = (
    region_df.dropna(subset=['rating'])
             .groupby('rating')['na_sales']
             .sum()
             .sort_values(ascending=False)
)
shares = na_rating / na_rating.sum() * 100
print("Ventas por rating ESRB — NA (con %)")
print((na_rating.to_frame()
       .assign(share_pct=shares)).reset_index())

# Barra
plt.figure(figsize=(8,4))
ax = na_rating.plot(kind='bar')
plt.title('Ventas por rating ESRB — NA')
plt.xlabel('Rating ESRB')
plt.ylabel('Ventas (millones USD)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Conclusiones Paso 4 — Perfil por región (breve)

NA
las Plataformas TOP: PS4 24.8%, XOne 21.3%, X360 18.7%, PS3 14.5%, 3DS 8.7% → Xbox pesa más que en EU.
Géneros TOP: Action 28.8%, Shooter 25.1%, Sports 14.9%, RPG 10.6%, Misc 6.3%.
ESRB: M 47.4%, E 22.7%, E10+ 15.6%, T 14.3% → fuerte sesgo a títulos maduros.

EU
las Plataformas TOP: PS4 36.0%, PS3 17.3%, XOne 13.2%, X360 10.8%, 3DS 7.9% → PlayStation domina más que en NA.
Géneros TOP: Action 30.1%, Shooter 22.4%, Sports 15.4%, RPG 9.4%, Racing 5.1%.
ESRB: M 37.0%, E 21.3%, E10+ 10.9%, T 10.7%.

JP
las Plataformas TOP: 3DS 48.2%, PS3 16.6%, PSV 13.2%, PS4 11.3%, WiiU 7.7% → portátiles/Nintendo muy fuertes; Xbox casi ausente.
Géneros TOP: RPG 36.3%, Action 28.8%, (después) Misc/Fighting/Shooter muy por atrás.
ESRB: patrón distinto a NA/EU; mayor peso de T/E que de M.

Resumen:
NA/EU prefieren M y consolas de salón; EU es más “PlayStation”, NA más balanceado con Xbox.
JP es “otra liga”: 3DS + RPG dominan; clasificación menos cargada a M.

## Pruebas de Hipotesis

In [ ]:
# Import para este paso

from scipy import stats

alpha = 0.05

In [ ]:
# H01: Medias de user_score en XOne y PC son iguales


# H0: mu_XOne = mu_PC
# H1: mu_XOne != mu_PC 

# Filtrar muestra 

gx = df_actual[(df_actual['platform'] == 'XOne') & (df_actual['user_score'].notna())]['user_score']
gp = df_actual[(df_actual['platform'] == 'PC')   & (df_actual['user_score'].notna())]['user_score']

print(f"Tamaños: XOne={gx.shape[0]}, PC={gp.shape[0]}")
print(f"Medias:  XOne={gx.mean():.3f}, PC={gp.mean():.3f}")

if gx.shape[0] >= 3 and gp.shape[0] >= 3:
    t_stat, p_val = stats.ttest_ind(gx, gp, equal_var=False)   # Welch
    print(f"t={t_stat:.3f}, p-value={p_val:.4f}, alpha={alpha}")
    if p_val < alpha:
        print("Conclusión: Rechazamos H0 > las medias son diferentes.")
    else:
        print("Conclusión: No rechazamos H0 > no hay evidencia de diferencia en medias.")
else:
    print("Muestra insuficiente para ejecutar la prueba con confianza.")



In [ ]:
# H01: Medias de user_score en Action y Sport son iguales


# H0: mu_Action = mu_Sport
# H1: mu_Action != mu_Sport 

# Filtrar muestra 

ga = df_actual[(df_actual['genre'] == 'Action')  & (df_actual['user_score'].notna())]['user_score']
gs = df_actual[(df_actual['genre'] == 'Sports')  & (df_actual['user_score'].notna())]['user_score']

print(f"Tamaños: Action={ga.shape[0]}, Sports={gs.shape[0]}")
print(f"Medias:  Action={ga.mean():.3f}, Sports={gs.mean():.3f}")

if ga.shape[0] >= 3 and gs.shape[0] >= 3:
    t_stat, p_val = stats.ttest_ind(ga, gs, equal_var=False)   # Welch
    print(f"t={t_stat:.3f}, p-value={p_val:.4f}, alpha={alpha}")
    if p_val < alpha:
        print("Conclusión: Rechazamos H0 → las medias son diferentes.")
    else:
        print("Conclusión: No rechazamos H0 → no hay evidencia de diferencia en medias.")
else:
    print("Muestra insuficiente para ejecutar la prueba con confianza.")
    

## Paso 5 — Pruebas de hipótesis 

H01 (XOne vs PC) — user_score
Hipótesis:
H0: mu_XOne = mu_PC
H1: mu_XOne != mu_PC  (bicaudal, alpha=0.05)
Resultados:
Tamaños: XOne=182, PC=155
Medias:  XOne=6.521, PC=6.270
Welch t-test: t=1.452, p=0.1476
Conclusión:
No rechazamos H0 > no hay evidencia de diferencia en la media de user_score entre XOne y PC.

Implicación práctica
La “calidad percibida por usuarios” es similar entre XOne y PC en 2013–2016.
Para 2017, no se puede esperar ventajas de satisfacción solo por la plataforma.
Dse determina que el PS4 domina EU; el Xbox domina más en NA), no por diferencia de user_score entre XOne y PC.

H02 (Action vs Sports) — user_score
Hipótesis:
H0: mu_Action = mu_Sports
H1: mu_Action != mu_Sports  (bicaudal, alpha=0.05)
Resultados:
Tamaños: Action=389, Sports=160
Medias:  Action=6.838, Sports=5.238
Welch t-test: t=10.233, p<0.0001
Conclusión:
Rechazamos H0 > sí hay diferencia; Action > Sports en user_score.

Implicación práctica
Los juegos de Action tienen, en promedio, mejor valoración de usuario que Sports.
Para 2017, priorizar títulos de Action, esto si se busca satisfacción del usuario y una mejor reseña.
El user_score alto no garantiza ventas; las críticas de expertos sí mostraron correlación positiva moderada con ventas (critics <> ventas ~ +0.406 en PS4).

## Conclusiones finales

Qué se hizo  
Cargamos `games.csv`, revisamos columnas y ausentes.  
Preparamos datos: columnas en minúsculas, `user_score`→float (tbd→NaN), `year_of_release`→Int64, creamos `total_sales`.  
Analizamos lanzamientos por año, ventas por plataforma, elegimos ventana **2013–2016**.  
Exploramos tendencias (curvas), distribución (boxplots), reseñas↔ventas, comparación entre plataformas y **perfil por región**.  
Probamos hipótesis (Welch t-test).

Hallazgos clave:
Plataformas (2013–2016): PS4 lidera en ventas; luego PS3, XOne, 3DS, X360. 2016 cae ligeramente.  
Distribución: ventas muy sesgadas (pocos “hits” empujan el promedio).  
Reseñas: en PS4, `critic_score` se asocia moderadamente con ventas (~+0.41); `user_score` ≈ sin relación.  
Regiones: NA/EU prefieren títulos M; EU más PlayStation, NA más balanceado con Xbox. JP: 3DS y RPG dominan.  
Multi-plataforma: ~16% de juegos venden más fuera de PS4 → la plataforma importa por título.

H01: XOne vs PC (user_score)
Resultado: p=0.1476 (>0.05) > no hay evidencia estadística de que la media de `user_score` sea distinta.
Traducción simple: la satisfacción promedio de usuarios es similar entre XOne y PC (2013–2016).
Implicación: no elijas plataforma por “gustó más a usuarios” entre estas dos. Decide por:
tamaño de mercado y presencia regional,
costos de port (tiempo/equipo),
catálogo/competencia y acuerdos comerciales.

H02: Action vs Sports (user_score)
Resultado: p<0.0001 > sí hay diferencia; medias: Action 6.84 > Sports 5.24.
Traducción simple: los juegos de Acción reciben mejores valoraciones de usuario que los de Deportes.
Implicación: si buscas satisfacción/recepción orgánica, Action es mejor apuesta que Sports.
Ojo: en PS4 vimos que critic_score (no user_score) tiene relación moderada con ventas. 
Usa user_score como señal de percepción, y critic_score como señal comercial.

Recomendaciones 2017

Plataformas por región
NA: PS4 + XOne (Xbox pesa más que en EU).
EU: PS4 dominante; XOne como complemento.
JP: 3DS y PS4 (Nintendo portátil muy fuerte).
Evita meter presupuesto en plataformas “legado” (PS3/X360) salvo relanzos baratos.

Géneros
Global: priorizacion de Action y Shooter (mejor tracción y ventas históricas).
JP: subir prioridad RPG (líder allí).
Cómo “evaluar” cada lanzamiento;
Usa un checklist para decidir budget y push:
critic_score esperado/previas (>75 como umbral de “push”).
Preórdenes / wishlists / registros de interés.
Encaje región–plataforma (RPG en JP, Shooter/Action en NA/EU).
Competencia y ventana (¿choca con un AAA?).
Costo de port y tiempo de salida multi–plataforma.
Señales de comunidad (seguimiento, trailers, menciones).
Decisión:
Si cumple 4–5 señales > Campaña fuerte (PR + ads + bundles).
Si cumple 2–3 >, Campaña media (segmentada por región).
Si cumple 2 o <, Lanzamiento discreto/long tail (tienda/promos puntuales).
Estrategia de portafolio:
No poner todo en un solo título. Distribuir riesgo:
“Apuestas foco” (2–3 títulos): alto presupuesto, previsión de ventas alta.
Mid-core (3–5): presupuesto medio, oportunidades regionales.
Long tail (varios): bajo costo, ingresos sostenidos (ofertas, bundles, temporadas).
Reglas prácticas:
Reasignar budget tras la primera semana: si un foco despega, añade inversión; si no, reduce y mueve a otro. Se recomienda mantener promos estacionales (Black Friday, Navidad, Golden Week JP).
Operativa de marketing

  